In [14]:
import os
import pandas as pd
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from tagger.core.mongo.models.order import Order
from pytz import timezone
from datetime import datetime, date
from tagger.models.inventory import Inventory, InventoryStatus
from tagger.models.shipping_notice import ShippingNotice, ShippingNoticeStatus

TZ = timezone("Asia/Seoul")

notice = ShippingNotice.objects.get(id=6)


CJ_COLUMNS = [
    "주문번호", "주문일", "수령자명", "수령자 우편번호", "수령자 연락처", "수령자주소", "상품명", "옵션명", "수량", "배송메모", "박스타입"
]



def get_cj_row(package):
    items = package.shipping_notice_items.all()
    
    item_name = items[0].item.name
    item_size = items[0].item.size
    
    item_count = len(items)
    if item_count > 1:
        item_name += f"외 {item_count - 1}"
        item_size += f"외 {item_count - 1}"
        
    return [
        package.code,
        date.today().isoformat().replace("-", "/"),
        package.recipient_name,
        package.postcode,
        f"{package.recipient_mobile[:3]}-{package.recipient_mobile[3:7]}-{package.recipient_mobile[7:11]}",
        package.address,
        item_name,
        item_size,
        1,
        items[0].item.extended_order.order.memo,
        "소"
    ]

def make_cj(orders):
    data = [get_cj_row(package) for package in notice.packages.all()]
            
    df = pd.DataFrame(data, columns=CJ_COLUMNS).sort_values(by="수령자명")
    df["수령자 우편번호"] = df["수령자 우편번호"].apply(lambda x: x.zfill(5))
    with open(f"../Downloads/cj_upload_{datetime.now().date().isoformat()}.xlsx", "wb") as file:
        df.to_excel(file, index=None)


In [8]:
for p in notice.packages.all():
    print(p.recipient_name)
    for item in p.shipping_notice_items.all():
        print(f"""ITEM {item.item.product_code} / INV {item.inventory.product_code}""")
        print(f"""CROSSCHECK {item.item.extended_order.order.orderstatus} {item.item.extended_order.order.payment.name}""")


정지영
ITEM MAXMARA___[MaxMara]WSDALIA___onesize / INV MAXMARA___[MaxMara]WSDALIA___onesize
CROSSCHECK DELIVERY_PREPARING [MaxMara] WSDALIA
손경희
ITEM SJSJ___언밸런스니트롱드레스___76(44) / INV SJSJ___언밸런스니트롱드레스___76(44)
CROSSCHECK DELIVERY_PREPARING 언밸런스 니트 롱 드레스 
김기량
ITEM MAXMARA___[택제거상품]Teca___42 / INV MAXMARA___[택제거상품]Teca___42
CROSSCHECK DELIVERY_PREPARING [택 제거 상품] Teca
김희경
ITEM MAXMARA___[WeekendMaxMara]PATNER(Camel)___42 / INV MAXMARA___[WeekendMaxMara]PATNER(Camel)___42
CROSSCHECK DELIVERY_PREPARING [Weekend MaxMara] PATNER(Camel)
이지오
ITEM BEANPOLEL___[빈폴스포츠]리브퀄팅구스다운점퍼(화이트)___90 / INV BEANPOLEL___[빈폴스포츠]리브퀄팅구스다운점퍼(화이트)___90
CROSSCHECK DELIVERY_PREPARING [빈폴스포츠] 리브 퀄팅 구스 다운 점퍼(화이트) 
김현주
ITEM BEANPOLEL___[빈폴스포츠]리브퀄팅구스다운점퍼(블랙)___95 / INV BEANPOLEL___[빈폴스포츠]리브퀄팅구스다운점퍼(블랙)___95
CROSSCHECK DELIVERY_PREPARING [빈폴스포츠] 리브 퀄팅 구스 다운 점퍼(블랙) 
장정임
ITEM BEANPOLEL___[빈폴스포츠](MEN)올라운더보아구스롱다운___100 / INV BEANPOLEL___[빈폴스포츠](MEN)올라운더보아구스롱다운___100
CROSSCHECK DELIVERY_PREPARING [빈폴스포츠] (MEN) 올라운더 보아 구스 롱 다운
김나나


In [15]:
make_cj(notice)
notice.status = ShippingNoticeStatus.SHIPPED
notice.save()

/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ShippingNotice.shipped_at received a naive datetime (2021-11-17 06:06:47.745142) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


'43D3W'

In [2]:
Inventory.objects.filter(out_order_id="618ccea526685ff0517561b5").first()

<Inventory: Inventory #167 [SHIPPED] 퀄팅 구스다운 점퍼  (211112-023-R)>